In [116]:
import numpy as np
from sklearn import svm
from sklearn import decomposition

In [117]:
dataset_train = np.loadtxt("/Users/neha/Documents/Kaggle/SkikitLearn/train.csv", delimiter=",")
dataset_label = np.loadtxt("/Users/neha/Documents/Kaggle/SkikitLearn/trainLabels.csv", delimiter=",")
dataset_test = np.loadtxt("/Users/neha/Documents/Kaggle/SkikitLearn/test.csv", delimiter=",")
dataset_train

array([[ 0.29940251, -1.22662419,  1.49842505, ..., -2.71555881,
        -2.68240859,  0.10105047],
       [-1.17417585,  0.33215734,  0.94991875, ...,  1.21321926,
         1.38293163, -1.81776106],
       [ 1.19222208, -0.41437073,  0.06705418, ...,  0.6564375 ,
        -0.93247282,  2.9874358 ],
       ..., 
       [ 0.91274769, -1.73403887, -1.04703517, ...,  0.06151283,
         0.8955364 ,  0.81368551],
       [ 2.43977998, -0.73551082, -0.90242559, ...,  0.22974842,
        -2.31108844, -3.4222166 ],
       [ 0.2289942 , -0.08545331,  0.87658248, ...,  0.40037229,
        -3.43103056,  2.37007995]])

In [118]:
pca = decomposition.PCA(n_components=30, whiten=True)
train_pca = pca.fit_transform(dataset_train)
test_pca = pca.transform(dataset_test)

In [119]:
print(pca.explained_variance_ratio_) 

[ 0.25054403  0.2055048   0.08026473  0.05033658  0.04895951  0.04489904
  0.04170781  0.03127937  0.02309827  0.02100166  0.0161973   0.01269925
  0.00885168  0.00846122  0.00827947  0.00817823  0.00808196  0.0078834
  0.00770056  0.00764196  0.00742926  0.00735226  0.00711202  0.00706576
  0.00691463  0.00684684  0.00663475  0.00646844  0.00632196  0.00625703]


In [120]:
#cumulative variance explained by first 30 principal components
round(sum(pca.explained_variance_ratio_),2)

0.96

In [121]:
##### implementation#####
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
train_pca, dataset_label, test_size=0.20, random_state=0)

In [122]:
svc_1 = svm.SVC(kernel='linear')

In [123]:
# splitting the data set into training and testing
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem
def evaluate_cross_validation(clf, X, y, K):
    #  # create a k-fold croos validation iterator
    cv = KFold(len(y), K, shuffle=True, random_state=0)    
# by default the score used is the one returned by score 
   # method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print scores
    print ("Mean score: {0:.3f} (+/-{1:.3f})").format(
        np.mean(scores), sem(scores))

evaluate_cross_validation(svc_1, X_train, y_train, 7)


[ 0.82608696  0.8173913   0.79824561  0.74561404  0.8245614   0.89473684
  0.84210526]
Mean score: 0.821 (+/-0.017)


In [124]:
from sklearn import metrics
def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    print "Accuracy on training set:"
    print round(clf.score(X_train, y_train),2)
    print "Accuracy on testing set:"
    print round(clf.score(X_test, y_test),2)
    y_pred = clf.predict(X_test)
    print "Classification Report:"
    print metrics.classification_report(y_test, y_pred)
    print "Confusion Matrix:"
    print metrics.confusion_matrix(y_test, y_pred)
      

In [125]:
train_and_evaluate(svc_1, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.84
Accuracy on testing set:
0.81
Classification Report:
             precision    recall  f1-score   support

        0.0       0.79      0.78      0.78        89
        1.0       0.82      0.84      0.83       111

avg / total       0.81      0.81      0.81       200

Confusion Matrix:
[[69 20]
 [18 93]]


In [126]:
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7)

In [127]:
evaluate_cross_validation(rbf_svc, X_train, y_train, 7)

[ 0.47826087  0.46956522  0.45614035  0.44736842  0.49122807  0.69298246
  0.44736842]
Mean score: 0.498 (+/-0.033)


In [128]:
train_and_evaluate(rbf_svc, X_train, X_test, y_train, y_test)

Accuracy on training set:
1.0
Accuracy on testing set:
0.45
Classification Report:
             precision    recall  f1-score   support

        0.0       0.45      1.00      0.62        89
        1.0       0.00      0.00      0.00       111

avg / total       0.20      0.45      0.27       200

Confusion Matrix:
[[ 89   0]
 [111   0]]


In [129]:
poly_svc = svm.SVC(kernel='poly', degree=3)
evaluate_cross_validation(poly_svc, X_train, y_train, 7)

[ 0.8173913   0.8173913   0.8245614   0.8245614   0.85964912  0.85964912
  0.84210526]
Mean score: 0.835 (+/-0.007)


In [130]:
train_and_evaluate(poly_svc, X_train, X_test, y_train, y_test)

Accuracy on training set:
0.98
Accuracy on testing set:
0.87
Classification Report:
             precision    recall  f1-score   support

        0.0       0.90      0.80      0.85        89
        1.0       0.85      0.93      0.89       111

avg / total       0.87      0.87      0.87       200

Confusion Matrix:
[[ 71  18]
 [  8 103]]


In [131]:
#Solution = np.dtype(bool)


In [132]:
Solution = poly_svc.predict(test_pca)
Solution

array([ 1.,  1.,  0., ...,  1.,  0.,  1.])

In [133]:
Id = np.arange(1,9001,1)
labels =  np.column_stack((Id, Solution))

In [134]:
np.savetxt('/Users/neha/Documents/Kaggle/SkikitLearn/output_1.csv',labels,delimiter=',')
